### Skeleton for the blog post
1. Introduction: Provide a motivation for calibration. Provide example applications where calibration matters. Explain calibration and sharpness to characterize accurate probabilistic forecast. 
2. Background: Explain the ideas behind quantile and distribution calibration.  
    1. Mathematical definitions 
    2. How does our idea compare with previous approaches to quantile/distribution calibration within this framework
    3. Evaluation of calibration
        - Revisit examples of proper loss functions
5. Recalibration as density estimation
    1. Explain the idea in simple words (no theoretical results)
    2. Provide and explain the main algorithm
    3. Provide/explain algorithms for specific use cases (quantile function regression and distribution calibrated classification)
6. Examples (worked out with our DistCal library code)
    1. Distribution calibrated regression
        - Bayesian Ridge regression as base model trained on the UCI housing regression dataset
        - Bayesian Neural Network as base model trained on the UCI housing regression dataset
        - Evaluation using check score and MAE/MSE
    2. Distribution calibrated classification 
        - Logistic Regression as base model trained on a UCI classification dataset
        - CNN as base model and MNIST/SVHN as the dataset
        - Evaluation using calibration score and accuracy
    
7. Conclusions and closing thoughts
